In [39]:
import numpy as np
import pandas as pd

import torch
import torchvision
from torchvision import transforms, models

from tqdm import tqdm
import os
import shutil

from PIL import Image

### Unzipping archive

In [ ]:
import zipfile

archive = "C:\\Users\н\Downloads\plates.zip"
with zipfile.ZipFile(archive, 'r') as zip_file:
    zip_file.extractall("C:\\Users\н\Desktop\jupyter_code\platesv2")

#### Transforming images to tensors (example)

trans = transforms.Compose([transforms.ToTensor()])

import glob
test = [trans(Image.open(img)).permute(1, 2, 0) for img in glob.glob("C://Users/н/Desktop/jupyter_code/platesv2/test/*.jpg")]   

### Separating into training and validation datasets

In [ ]:
os.mkdir('val')
os.makedirs("C://Users/н/Desktop/jupyter_code/platesv2/val/cleaned")
os.makedirs("C://Users/н/Desktop/jupyter_code/platesv2/val/dirty")

In [ ]:
class_names = ['cleaned', 'dirty']

data_root = "C://Users/н/Desktop/jupyter_code/platesv2/train"
dest_root = "C://Users/н/Desktop/jupyter_code/platesv2/val"

for class_name in class_names:
    file_source = os.path.join(data_root, class_name)
    print(file_source)
    for i, file_name in enumerate(tqdm(os.listdir(file_source))):
        if i % 6 == 0:
            file_dest = os.path.join(dest_root, class_name) 
            shutil.move(file_source + '/' + file_name, file_dest)

### Making some transformations to expand train set

In [42]:
train_transforms1 = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(1),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

train_transforms2 = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

train_transforms3 = transforms.Compose([
    transforms.RandomRotation(degrees=90, fill=255),
    transforms.CenterCrop(180),
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ColorJitter(hue=(0.1, 0.2)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

### Creating dataloaders

In [43]:
batch_size = 8

In [56]:
train_dir = "C://Users/н/Desktop/jupyter_code/platesv2/train"
val_dir = "C://Users/н/Desktop/jupyter_code/platesv2/val"

tr_trans = [train_transforms3] #train_transforms1, train_transforms2, train_transforms4, train_transforms5, train_transforms6,
           #train_transforms7, train_transforms8, train_transforms9, train_transforms10, train_transforms11,
           #train_transforms12]


train_dataset = torchvision.datasets.ImageFolder(train_dir, train_transforms1)
for t_trans in tr_trans:
    train_dataset += torchvision.datasets.ImageFolder(train_dir, t_trans)

val_dataset = torchvision.datasets.ImageFolder(val_dir, val_transforms)

In [57]:
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=batch_size)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=batch_size)

### Defining model

In [58]:
model = models.resnet50(pretrained=True)

for param in model.parameters():
    param.requires_grad = False
    
for param in model.fc.parameters():
    param.requires_grad = True

for param in model.avgpool.parameters():
    param.requires_grad = True
    
for param in model.layer4.parameters():
    param.requires_grad = True
    
model.fc = torch.nn.Sequential(
            torch.nn.Linear(model.fc.in_features, 256),
            torch.nn.ReLU(),
            torch.nn.Linear(256, 2))

loss = torch.nn.CrossEntropyLoss()
#optimizer = torch.optim.Adam(model.parameters(), amsgrad=True, lr=1.0e-3)
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=1.1)

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

### Training model

In [41]:
def train_model(model, loss, optimizer, scheduler, num_epochs):
    for epoch in range(num_epochs):
        print(f'Epoch {epoch + 1}/{num_epochs}:')
        for phase in ['train', 'val']:
            if phase == 'train':
                optimizer.step()
                dataloader = train_dataloader
                scheduler.step()
                model.train()
            else:
                dataloader = val_dataloader
                model.eval()
            
            running_loss = 0.
            running_acc = 0.
            
            for inputs, labels in tqdm(dataloader):
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'train'):
                    preds = model(inputs)
                    loss_value = loss(preds, labels)
                    preds_class = preds.argmax(dim=1)

                    if phase == 'train':
                        loss_value.backward()
                        optimizer.step()
            
                running_loss += loss_value.item()
                running_acc += (preds_class == labels.data).float().mean()
            
            epoch_loss = running_loss / len(dataloader)
            epoch_acc = running_acc / len(dataloader)
        
    return model

In [59]:
train_model(model, loss, optimizer, scheduler, num_epochs=30)

Epoch 1/30:


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.10s/it]

Epoch 2/30:



100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.44s/it]

Epoch 3/30:


...

Epoch 29/30:



100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.23s/it]

Epoch 30/30:



100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.22s/it]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

============================================================================================================================

### Preparing test set

In [ ]:
test_source = "C://Users/н/Desktop/jupyter_code/platesv2/test"
os.makedirs("C://Users/н/Desktop/jupyter_code/platesv2/test/unknown")

for i, file_name in enumerate(tqdm(os.listdir(test_source))):
        file_dest = "C://Users/н/Desktop/jupyter_code/platesv2/test/unknown"
        shutil.move(test_source + '/' + file_name, file_dest)

In [47]:
class TestImagesWithPaths(torchvision.datasets.ImageFolder):
    def __getitem__(self, index):
        original_tuple = super(TestImagesWithPaths, self).__getitem__(index)
        path = self.imgs[index][0]
        tuple_with_path = (original_tuple + (path, ))
        
        return tuple_with_path

In [48]:
test_source = "C://Users/н/Desktop/jupyter_code/platesv2/test"

test_dataset = TestImagesWithPaths(test_source, val_transforms)

test_dataloader = torch.utils.data.DataLoader(
    test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

### Making predictions

In [49]:
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [50]:
test_predictions = []
test_img_paths = []

for inputs, labels, paths in tqdm(test_dataloader):
    with torch.set_grad_enabled(False):
        preds = model(inputs)
    test_predictions.append(torch.nn.functional.softmax(preds, dim=1)[:,1].data.cpu().numpy())
    test_img_paths.extend(paths)

test_predictions = np.concatenate(test_predictions) 

100%|██████████████████████████████████████████████████████████████████████████████████| 93/93 [01:05<00:00,  1.41it/s]


In [51]:
def get_submission_df(thresh):
    submission_df = pd.DataFrame.from_dict({'id': test_img_paths, 'label': test_predictions})
    
    submission_df['label'] = submission_df['label'].map(lambda pred: 'dirty' if pred > thresh else 'cleaned')
    submission_df['id'] = submission_df['id'].str.replace('C://Users/н/Desktop/jupyter_code/platesv2/test', '')
    submission_df['id'] = submission_df['id'].str.replace('.jpg', '')
    submission_df['id'] = submission_df['id'].str.replace('unknown', '')
    submission_df['id'] = submission_df['id'].str.replace('\\', '')
    
    submission_df.set_index('id', inplace=True)
    submission_df.to_csv('my_submission.csv')
    
    return submission_df

In [55]:
submission_df = get_submission_df(thresh=0.1)

C:\Users\н\AppData\Local\Temp\ipykernel_13948\2184212342.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  submission_df['id'] = submission_df['id'].str.replace('.jpg', '')
C:\Users\н\AppData\Local\Temp\ipykernel_13948\2184212342.py:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  submission_df['id'] = submission_df['id'].str.replace('\\', '')
